In [71]:
""" read from a SQLite database and return data to templates """

from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy
from flask_bootstrap import Bootstrap
from sqlalchemy.sql import text
from sqlalchemy import create_engine

app = Flask(__name__)

# Flask-Bootstrap requires this line
Bootstrap(app)

# the name of the database; add path if necessary
db_name = 'recipes_data.db'

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + db_name

app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = True

# this variable, db, will be used for all SQLAlchemy commands
db = SQLAlchemy(app)

# each table in the database needs a class to be created for it
# db.Model is required - don't change it
# identify all columns by name and data type
class Recipe(db.Model):
    __tablename__ = 'recipes'
    id = db.Column(db.Integer, primary_key=True)
    timestamp_entered = db.Column(db.String)
    timestamp_prepared = db.Column(db.String)
    URL = db.Column(db.String)
    title = db.Column(db.String)
    keywords = db.Column(db.String)
    ingredients = db.Column(db.String)
    instructions = db.Column(db.String)
    notes = db.Column(db.String)
    make_again = db.Column(db.Integer)
    things_to_try = db.Column(db.String)
    image = db.Column(db.String)
    category = db.Column(db.String)

In [10]:
categories = list(Recipe.query.with_entities(Recipe.category).distinct())

In [22]:
list(Recipe.query.with_entities(Recipe.category).distinct())[0].category

'Discrete'

In [23]:
categories = Recipe.query.with_entities(Recipe.category).distinct()
categories_list = [c.category for c in categories]

In [83]:
recipe_data = db.session.query(Recipe.title,Recipe.id).from_statement(text("SELECT * from recipes where ingredients like '%tofu%'")).all()

In [86]:
[x.id for x in recipe_data]

[2, 4]

In [88]:
Recipe.query.from_statement(text("SELECT * from recipes where ingredients like '%tofu%'")).all()

[<Recipe 2>, <Recipe 4>]

In [74]:
engine = create_engine('sqlite:///recipes_info.db')
with engine.connect() as con:

    rs = con.execute('SELECT title FROM recipes')

    for row in rs:
        print(row)

OperationalError: (sqlite3.OperationalError) no such table: recipes
[SQL: SELECT title FROM recipes]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [54]:
ingredients = [i.ingredients for i in Recipe.query.with_entities(Recipe.ingredients).distinct()]
a = [x.split(',') for x in ingredients]
ingredients_list = sorted([x.strip() for x in list(itertools.chain.from_iterable(a))])

In [55]:
ingredients_list

['a whole lotta gumption',
 'beets',
 'breading shit',
 'literally just some garbage',
 'tofu',
 'vital wheat gluten']

In [ ]:
#routes
@app.route('/')
def index():
    try:
        categories = Recipe.query.with_entities(Recipe.category).distinct()
        return render_template('index.html', categories=categories)
    except Exception as e:
        # e holds description of the error
        error_text = "<p>The error:<br>" + str(e) + "</p>"
        hed = '<h1>Something is broken.</h1>'
        return hed + error_text


@app.route('/inventory/<category>')
def inventory(category):
    try:
        recipes = Recipe.query.filter_by(category=category).order_by(Recipe.title).all()
        return render_template('list.html', recipes=recipes, category=category)
    except Exception as e:
        # e holds description of the error
        error_text = "<p>The error:<br>" + str(e) + "</p>"
        hed = '<h1>Something is broken.</h1>'
        return hed + error_text


if __name__ == '__main__':
    app.run(debug=True)